<!--
#  Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.
#
#    Licensed under the Apache License, Version 2.0 (the "License").
#    You may not use this file except in compliance with the License.
#    You may obtain a copy of the License at
#
#        http://www.apache.org/licenses/LICENSE-2.0
#
#    Unless required by applicable law or agreed to in writing, software
#    distributed under the License is distributed on an "AS IS" BASIS,
#    WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#    See the License for the specific language governing permissions and
#    limitations under the License.
-->

# Sample notebook to build a Jupyter Image with GPU enabled.

## Content
1. [Configuration](#Configuration)
1. [Preparation](#Preparation)
1. [Build Image](#Build-Image)
1. [Running Container using the profile](#Running-container)


### Configuration

In [ ]:
image_name = 'gpu-jupyter-user'
folder_name = 'aws-orbit_jupyter-user'

### Preparation
The standard `jupyter-user` image we deploy contains a copy of the source files we use to build the image. These sources can be used to build and deploy customized images. Here we demonstrate how to use these sources to build a custom image with an alternative `BASE_IMAGE` that is preconfigured with GPU libraries.

In [ ]:
archive = f'{folder_name}.tar.gz'

!rm -r ./$folder_name

!cp /opt/orbit/$archive ./$archive

!tar zxvf ./$archive 

### Build Image

Lets see how  orbit build image works...

In [ ]:
!orbit build image --help

get our orbit env and team names

In [ ]:
env_name = %env AWS_ORBIT_ENV
team_name = %env AWS_ORBIT_TEAM_SPACE
user_name = %env USERNAME
namespace = %env AWS_ORBIT_USER_SPACE
(env_name,team_name, user_name, namespace)

Repository name will be created from the image name prefixed by the env context.  Users are only able to manipulate ECR repos that start with 'orbit-{env_name}/users/'

In [ ]:
repository_name = (f"orbit-{env_name}/users/{image_name}")
repository_name

In [ ]:
!aws ecr delete-repository --repository-name $repository_name --force

In [ ]:
pwd = %pwd
pwd

Now lets run the command.

**_This can take some time (~20 minutes)._**

In [ ]:
%%time

output = !orbit build image -e $env_name -d $pwd/$folder_name -n $image_name -s bundle.sh --timeout 45 --build-arg BASE_IMAGE=cschranz/gpu-jupyter
output

Lets get the image address from the output of the previous command

In [ ]:
look_for = 'ECR Image Address='
image = None
for o in output:
    if look_for in o:
        image = o[o.index(look_for) + len(look_for):]
        print(image)

assert(image != None)       
    

In [ ]:
# check that the image was built
import json
print(repository_name)
images = !aws ecr list-images --repository-name $repository_name
images = "".join(images)
im = json.loads(images)
print(im['imageIds'])
assert(len(im['imageIds']) > 0)

### Building the PodSetting for the Image

In [ ]:
import json

customnameGPU = "orbit-gpu-image-sample-"+team_name
description= " Machine Learning Image GPU - 2 CPU + 4G MEM + 1 GPU "
podsetting={
            "name": customnameGPU,
            "description": description,
            "image": image,
            "resources":{
                "limits":{            
                    "cpu": "2.0",
                    "memory": "4Gi"
                    "nvidia.com/gpu": "1"
                    },
                "requests":{
                    "cpu": "2.0",
                    "memory": "4Gi"
                    "nvidia.com/gpu": "1"
                    }
            },
            "node-group":"primary-gpu",
            "env":[
                {
                    "name":"source",
                    "value":"regressiontests"
                }
            ]
        }

with open("podsetting_data_gpu.json", 'w') as f:
    json.dump(podsetting, f)

### NOTE: "node-group":"primary-gpu" can be replaced with "instance-type":"g4dn.xlarge" 

In [ ]:
!orbit build podsetting --help 

In [ ]:
!orbit build podsetting --debug -e $env_name -t $team_name  podsetting_data_gpu.json

### Running container

Lets run a container using the profile and image we created

In [ ]:
folder_name='gpu'
%cd ~/shared/samples/notebooks/I-Image/$folder_name

In [ ]:
pwd = %pwd
pwd

In [ ]:
import json
run = {
      "compute": {
          "container" : {
              "p_concurrent": "1"
          },
          "node_type": "ec2",
          "podsetting":customnameGPU
      },
      "tasks":  [{
          "notebookName": "test-image.ipynb",
          "sourcePath": pwd,
          "targetPath": f"/home/jovyan/shared/regression/notebooks/I-Image/{folder_name}",
          "params": {
          }
        }]
 }

with open("run.json", 'w') as f:
    json.dump(run, f)

In [ ]:
%%time

!orbit run notebook --env $env_name --team $team_name --user testing --wait --tail-logs run.json

***

## To Delete the podsetting, your command would look like this


!orbit delete podsetting -n {customname} -t {team_name} --debug

## OPTIONAL: You can use kubectl to load the podsettings as indicated below 

### (convert the cells to code from markdown)

import json

customnameGPU = "orbit-gpu-image-sample-"+team_name

with open("podsetting_gpu.yaml", "w") as file:
    file.write("""
    kind: PodSetting
    apiVersion: orbit.aws/v1
    metadata:
      labels:
        orbit/env: {env_name}
        orbit/space: team
        orbit/team: {team_name}
      name: {customname}
      namespace: {team_name}
    spec:
      containerSelector:
        jsonpath: metadata.labels.app
      desc: Machine Learning Image GPU - 2 CPU + 4G MEM + 1 GPU
      env:
        - name: custom_name
          value: custom_value
      image: >-
        {image}
      nodeSelector:
        orbit/node-group: primary-gpu
      podSelector:
        matchExpressions:
          - key: orbit/{customname}
            operator: Exists
      resources:
        limits:
          cpu: '2.0'
          memory: 4Gi
          nvidia.com/gpu: '1'
        requests:
          cpu: '2.0'
          memory: 4Gi
          nvidia.com/gpu: '1'
      securityContext:
        runAsUser: 1000
    """.format(team_name=team_name,env_name=env_name,customname=customnameGPU,image=image)
)

!kubectl apply -f podsetting_gpu.yaml -n {team_name}

import json

customnameVGPU = "orbit-vgpu-image-sample-"+team_name

with open("podsetting_vgpu.yaml", "w") as file:
    file.write("""
    kind: PodSetting
    apiVersion: orbit.aws/v1
    metadata:
      labels:
        orbit/env: {env_name}
        orbit/space: team
        orbit/team: {team_name}
      name: {customname}
      namespace: {team_name}
    spec:
      containerSelector:
        jsonpath: metadata.labels.app
      desc: Machine Learning Image vGPU - 2 CPU + 4G MEM + 1 GPU
      env:
        - name: custom_name
          value: custom_value
      image: >-
         {image}
      nodeSelector:
        node.kubernetes.io/instance-type: g4dn.xlarge
      podSelector:
        matchExpressions:
          - key: orbit/{customname}
            operator: Exists
      resources:
        limits:
          cpu: '2.0'
          k8s.amazonaws.com/vgpu: '1'
          memory: 4Gi
        requests:
          cpu: '2.0'
          k8s.amazonaws.com/vgpu: '1'
          memory: 4Gi
      securityContext:
        runAsUser: 1000
    """.format(team_name=team_name,env_name=env_name,customname=customnameVGPU,image=image)
)

!kubectl apply -f podsetting_vgpu.yaml -n {team_name}